In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import  EarlyStopping
from sklearn.model_selection import StratifiedKFold 

In [ ]:
features = [f'f_{i}' for i in range(300)]
data_types_dict = {
    'time_id': 'int32',
    'investment_id': 'int16',
    "target": 'float16',
}

for f in features:
    data_types_dict[f] = 'float16'

In [ ]:
features += ['investment_id','time_id']

In [ ]:
%%time
train = pd.read_csv('/kaggle/input/ubiquant-market-prediction/train.csv', 
                       usecols = data_types_dict.keys(),
                       dtype=data_types_dict, 
                   nrows= 1000000
                       )

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
seed = 0
folds = 10
models = []
epochs = 500
target = 'target'
train = train.head(1000000)
skf = StratifiedKFold(folds, shuffle = True, random_state = seed)
callbacks =  [EarlyStopping(patience=3,monitor="mse",mode="min")]
for train_index, test_index in skf.split(train, train['investment_id']):
    train = train.iloc[train_index]
#     valid = train.iloc[test_index]
    
    model = Sequential([
        Dense(512,activation='relu',input_dim=302),
        Dense(256,activation='relu'),
        Dense(128,activation='relu'),
        Dense(256,activation='relu'),
        Dense(128,activation='relu'),
        Dense(1,activation=keras.activations.linear)])
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mse'])
    model.fit(train[features], train[target],epochs=epochs,validation_split = 0.2)
    models.append(model)

In [ ]:
# del train

In [ ]:
import ubiquant
env = ubiquant.make_env()   
iter_test = env.iter_test()  

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    
    test_df['target']  = 0
    
    for model in models:
        test_df['target'] += model.predict(test_df[features])
    test_df['target'] /= len(models)
    env.predict(test_df[['row_id','target']])